### Import Libraries

In [1]:
import arcpy
import pandas as pd
import numpy as np
import os
import censusdata as cs
from census import Census
pd.set_option('display.max_rows',None, 'display.max_columns',None,'display.max_colwidth', None)

## Set Directory

In [2]:
path = 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness'
os.chdir(path)

In [3]:
ProjectGDB =  r"C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\"
arcpy.env.overwriteOutput = True
arcpy.env.workspace = ProjectGDB

- Residential Permits

In [4]:
ResidentialPermits = arcpy.management.SelectLayerByAttribute(
    "Issued_Permits",
    "NEW_SELECTION",
    """ SUBDESC IN ('Single-Family','Two-Family','Residential','Residential Pool Barrier','Residential Privacy',
        'Residential Spa','Residential Swimming Pool','Garage Conversion','Detached Garage',
        'Permanent Storage/Accy Building (over 320 sq ft)','Portable Storage/Accy Building (320 sq ft and under)',
        'Patio Cover/Gazebo','Carport','Retaining Wall' )
        
    AND WORKDESC IN ('Addition','Demolition', 'Foundation','Interior Finish','Move','New','New Construction',
    'New Construction - Shell','Remodel/Alterations/Repairs','Remove/Replace','Repair', 'Repair / Replacement','Reroof',
    'Solar Panel', 'Window/Door Replacement')
    """)

ResidentialPermcount = int(arcpy.management.GetCount(ResidentialPermits)[0])
print(ResidentialPermcount)

10463


In [5]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable(ResidentialPermits)

analysisfields = ['FOLDERYEAR', 'STATUSDESC', 'ISSUEDATE', 'FINALDATE', 'SUBDESC', 'FOLDERNAME',
                  'Constructi','MainUse','LandUseDes','Structure','Census','NameofBusi','SignConstr',
                  'ServiceGri','FOLDERCOND','ImportDate']

for field in fieldmapping.fields:
    if field.name not in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        
if arcpy.Exists('ResidentialPermits'):
    arcpy.Delete_management('ResidentialPermits')

arcpy.conversion.ExportFeatures(ResidentialPermits,"ResidentialPermits","","NOT_USE_ALIAS",fieldmapping,None)

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ResidentialPermits'>

In [6]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonMarketAssessmentAreasEnriched")
fieldmapping.addTable("ResidentialPermits")

analysisfields = ["ImportDate","FOLDERYEAR","STATUSDESC","ISSUEDATE","FINALDATE","SUBDESC",
                  "FOLDERNAME", "Constructi","MainUse","LandUseDes","Structure","Census","NameofBusi",
                  "SignConstr","ServiceGri","FOLDERCOND"]

for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonMarketAssessmentAreasEnriched",
                           "ResidentialPermits","ArlingtonResNearby",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonResNearby","Join_Count","NearbyResidentialDevelopment","NearbyResidentialDevelopment")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonResNearby'>

- Commercial Permits

In [7]:
CommercialPermits = arcpy.management.SelectLayerByAttribute(
 "Issued_Permits",
    "NEW_SELECTION",
    """
    SUBDESC IN ('Assembly','Awning', 'Billboard', 'Boat Dock', 'Building Identification Sign', 'Business',
    'Canopy / Fascia Sign', 'Clean and Show', 'Commercial Privacy', 'Commercial Screening', 'Decks',
    'Directional',  'Directory',  'Educational',  'Entryway Sign',  'Existing Business/New Owner',
    'Expanding Lease Space',  'Factory',  'Ground',  'Institutional', 'Manufactured Housing',
    'Marquee Sign','Menu Board', 'Mercantile',  'Multi Tenant Ground Sign', 'New Tenant',
    'Onsite Development', 'Other Undefined Sign', 'Pole Sign','Projecting (Blade) Sign',
    'Same Business Owner/New Business Name','Shell Building/House Lights', 'Storage',
    'Subdivision Screening Wall', 'Temporary Construction Office', 'Temporary Sales Office',
    'Utility & Miscellaneous', 'Wall Sign'
    )
    AND WORKDESC IN ( 'New','New Construction','New Construction - Shell','Interior Finish',
    'Remodel/Alterations/Repairs', 'Repair', 'Repair / Replacement','Remove/Replace', 'Reroof',
    'Face Change', 'Foundation', 'Solar Panel', 'Window/Door Replacement','Demolition' )
    """
)

CommercialPermcount = int(arcpy.management.GetCount(CommercialPermits)[0])
print(CommercialPermcount)

4595


In [8]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable(CommercialPermits)

analysisfields = ['FOLDERYEAR', 'STATUSDESC', 'ISSUEDATE', 'FINALDATE', 'SUBDESC', 'FOLDERNAME',
                  'Constructi','MainUse','LandUseDes','Structure','Census','NameofBusi','SignConstr',
                  'ServiceGri','FOLDERCOND','ImportDate']

for field in fieldmapping.fields:
    if field.name not in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        
if arcpy.Exists('CommercialPermits'):
    arcpy.Delete_management('CommercialPermits')

arcpy.conversion.ExportFeatures(CommercialPermits,"CommercialPermits","","NOT_USE_ALIAS",fieldmapping,None)

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\CommercialPermits'>

In [9]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonResNearby")
fieldmapping.addTable("CommercialPermits")

analysisfields = ["ImportDate","FOLDERYEAR","STATUSDESC","ISSUEDATE","FINALDATE","SUBDESC",
                  "FOLDERNAME", "Constructi","MainUse","LandUseDes","Structure","Census","NameofBusi",
                  "SignConstr","ServiceGri","FOLDERCOND"]

for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonResNearby",
                           "CommercialPermits","ArlingtonComNearby",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonComNearby","Join_Count","NearbyCommercialDevelopment","NearbyCommercialDevelopment")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonComNearby'>

In [10]:
NewConstruction = arcpy.management.SelectLayerByAttribute(
 "Issued_Permits",
    "NEW_SELECTION",
    """
    WORKDESC IN ('New Construction - Shell', 'New Construction')
    """
)

NewConstructionCount = int(arcpy.management.GetCount(NewConstruction)[0])
print(NewConstructionCount)

3243


In [11]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable(NewConstruction)

analysisfields = ['FOLDERYEAR', 'STATUSDESC', 'ISSUEDATE', 'FINALDATE', 'SUBDESC', 'FOLDERNAME',
                  'Constructi','MainUse','LandUseDes','Structure','Census','NameofBusi','SignConstr',
                  'ServiceGri','FOLDERCOND','ImportDate']

for field in fieldmapping.fields:
    if field.name not in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        
if arcpy.Exists('NewConstruction'):
    arcpy.Delete_management('NewConstruction')

arcpy.conversion.ExportFeatures(NewConstruction,"NewConstruction","","NOT_USE_ALIAS",fieldmapping,None)

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\NewConstruction'>

In [12]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable("ArlingtonComNearby")
fieldmapping.addTable("NewConstruction")

analysisfields = ["ImportDate","FOLDERYEAR","STATUSDESC","ISSUEDATE","FINALDATE","SUBDESC",
                  "FOLDERNAME", "Constructi","MainUse","LandUseDes","Structure","Census","NameofBusi",
                  "SignConstr","ServiceGri","FOLDERCOND"]

for field in fieldmapping.fields:
    if field.name in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

arcpy.analysis.SpatialJoin("ArlingtonComNearby",
                           "NewConstruction","ArlingtonMarketReadiness",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",fieldmapping,
                           "WITHIN_A_DISTANCE",
                           "1 Miles",
                           "",
                           None)

arcpy.management.AlterField("ArlingtonMarketReadiness","Join_Count","NewConstructionActivity","NewConstructionActivity")

<Result 'C:\\Users\\Thepr\\Documents\\Consulting\\Market Readiness\\Market Readiness.gdb\\ArlingtonMarketReadiness'>

### Proximity to Highway